Merging and Joining

Inner Join on Chinook Database

In [1]:
import pandas as pd
import sqlite3

In [4]:

conn = sqlite3.connect('chinook.db')

customers = pd.read_sql('SELECT * FROM customers', conn)
invoices = pd.read_sql('SELECT * FROM invoices', conn)

customer_invoices = pd.merge(customers,invoices,on='CustomerId',how='inner')

invoice_counts = customer_invoices.groupby('CustomerId')['InvoiceId'].count().reset_index()
invoice_counts.columns = ['CustomerId', 'InvoiceCount']

conn.close()

Outer Join on Movie Data

In [7]:
movie=pd.read_csv('movie.csv')
director_color=movie[['color','director_name']].drop_duplicates()
director_reviews = movie[['director_name', 'num_critic_for_reviews']]

left_join = pd.merge(director_color, director_reviews, on='director_name', how='left')
outer_join = pd.merge(director_color, director_reviews, on='director_name', how='outer')

join_counts=pd.DataFrame({
    'Join Type':['Left Join','Outer Join'],
    'Count':[left_join.shape[0],outer_join.shape[0]]
})
join_counts

,Join Type,Count
0,Left Join,5761
1,Outer Join,5761


Grouping and Aggregating

Grouped Aggregations on Titanic

In [11]:
titanic=pd.read_excel('titanic.xlsx')

class_stats = titanic.groupby('Pclass').agg(
    AverageAge=('Age', 'mean'),
    TotalFare=('Fare', 'sum'),
    PassengerCount=('PassengerId', 'count')
).reset_index()

class_stats_df = pd.DataFrame(class_stats)
class_stats_df

,Pclass,AverageAge,TotalFare,PassengerCount
0,1,38.233441,18177.4125,216
1,2,29.877630,3801.8417,184
2,3,25.140620,6714.6951,491


Multi-level Grouping on Movie Data

In [12]:
movie_groups=movie.groupby(['color','director_name']).agg(
    Total_statist=('num_critic_for_reviews','sum'),
    Average_duration=('duration','mean')
).reset_index()
movie_groups
#

,color,director_name,Total_statist,Average_duration
0,Black and White,Akira Kurosawa,153.0,202.0
1,Black and White,Aleksey German,121.0,177.0
2,Black and White,Alex Garland,489.0,108.0
3,Black and White,Alexander Payne,433.0,115.0
4,Black and White,Alfred Hitchcock,434.0,119.0
...,...,...,...,...
2485,Color,Zoran Lisinac,17.0,108.0
2486,Color,Álex de la Iglesia,71.0,104.0
2487,Color,Émile Gaudreault,67.0,92.0
2488,Color,Éric Tessier,9.0,99.0


Nested Grouping on Flights

In [13]:
# Load flights data
flights = pd.read_csv('flights.csv')

# Group by Year and Month
flight_stats = flights.groupby(['Year', 'Month']).agg(
    TotalFlights=('FlightNumber', 'count'),
    AverageArrivalDelay=('ArrDelay', 'mean'),
    MaxDepartureDelay=('DepDelay', 'max')
).reset_index()

FileNotFoundError: [Errno 2] No such file or directory: 'flights.csv'

Applying Functions

Apply a Custom Function on Titanic

In [15]:
def classify_age(age):
    if pd.isnull(age):
        return 'Unknown'
    return 'Child' if age < 18 else 'Adult'

titanic['Age_Group'] = titanic['Age'].apply(classify_age)

classify_age(30)

'Adult'

Normalize Employee Salaries

In [17]:
employee=pd.read_csv('employee.csv')
def normalize_salary(salary):
    mean_salary = employee['Salary'].mean()
    std_salary = employee['Salary'].std()
    normalized_salary = (salary - mean_salary) / std_salary
    return normalized_salary



Custom Function on Movies

In [20]:
def custom(duration):
  if pd.isnull(duration):
    return 'Unknown'
  if duration<60:
    return 'Short'
  elif duration<=120:
    return 'Medium'
  else:
    return 'Long'

movie['Duration_Category']=movie['duration'].apply(custom)

custom(100)


'Medium'

Using pipe

Pipeline on Titanic

In [21]:
def titanic_pipeline(df):
    df = df.query('Survived == 1')

    df['Age'] = df['Age'].fillna(df['Age'].mean())

    df['Fare_Per_Age'] = df['Fare'] / df['Age']

    return df

titanic_processed = titanic_pipeline(titanic.copy())

In [ ]:
def flights_pipeline(df):
    # Filter flights with departure delay > 30 minutes
    df = df.query('DepDelay > 30')

    # Calculate delay per hour (assuming AirTime is in minutes)
    df['Delay_Per_Hour'] = df['DepDelay'] / (df['AirTime'] / 60)

    return df

flights_processed = flights_pipeline(flights.copy())